In [ ]:
# preliminary stuff - get demo BIDS dataset
!datalad install ///workshops/nipype-2017/ds000114
!datalad remove ds000114/derivatives/*
!datalad get -r ds000114
# also double up import to avoid ugly warnings printing during slideshow
from bids.layout import BIDSLayout
import os.path
import shutil
# house keeping
if os.path.exists("output"):
    shutil.rmtree("output", ignore_errors=True)

In [ ]:
# and demo dicoms for dcm2bids
!datalad install -g ///dicoms/dartmouth-phantoms/bids_test5-20170120

# <center>Brain Imaging Data Structure (BIDS): A standard format for neuroscience data</center>
<center>Johan Carlin</center>
<p></p>
<center><a href>https://github.com/jooh/notebook_bids_OSD2018</a></center>
<center><img src="MRC_CBU_Cambridge_colour_web_A5.png"></center>

## In BIDS 1.1.1
* MRI - anatomical, functional, diffusion, field maps…
* MEG
* Physiological recordings
* Behavioural data

## Draft extensions
* Model specification
* Derivatives - structural, functional, resting state, diffusion…
* New modalities - EEG, PET, ASL, Eye tracking, intracranial EEG

# MRI example

In [ ]:
# basic directory structure
!tree -L 2 ds000114

In [ ]:
# and inside one func folder
subdir = "ds000114/sub-01/func"
!ls -1 {subdir}
# and an events file
import pandas as pd
pd.read_csv(
    f"{subdir}/sub-01_task-linebisection_events.tsv",delimiter="\t")

# Why BIDS?

**Users** get easy access to new analysis tools, e.g.
`fmriprep ds000114 outdir sub-01 -w workdir`

In [ ]:
# Developers:
from bids.layout import BIDSLayout
layout = BIDSLayout("ds000114")
subjects = layout.get_subjects()
print(subjects)
layout.get(subject=subjects[0], type="T1w", return_type='file')

# Dcm2Bids
Fully automated conversion from dicom to BIDS-compliant output data.
* Converts dicoms to nifti (using dcm2niix), optionally anonymises structurals
* Moves niftis and header sidecars to the correct BIDS-format locations
* Adds custom fields to header sidecars as needed for BIDS compliance
* Initialises all required BIDS project files with sensible defaults

# Input dicoms

In [ ]:
!tree -L 1 bids_test5-20170120/phantom-1/

# Minimal configuration for a first test run
dcm2bids uses a configuration JSON file to map from dicom series to BIDS output files

In [ ]:
# you could just type it out but we we will extend the dict below
import json
config = dict(descriptions=[])
def writeconfig(config, filename="config.json"):
    with open(filename, "w") as fp:
        json.dump(config, fp, indent=4)
writeconfig(config)
!cat config.json

In [ ]:
# First test run of dcm2bids
!dcm2bids -d bids_test5-20170120 -p 01 -c config.json -o output

In [ ]:
# Unsurprisingly there's not much in the output dir
# but NB the tmp_dcm2bids dir!
!tree output

In [ ]:
# so the trick is to extend the config json to unambiguously identify our target
# acquisitions. Let's start with the t1. What's in the sidecar?
!cat output/tmp_dcm2bids/sub-01/bids_test5-20170120_anat_T1w_acq-MPRAGE_run+_20170120084340_series005.json

In [ ]:
# so our first description might be
config["descriptions"] = []
config["descriptions"].append({"dataType": "anat", 
                               "suffix": "T1w",
                               "criteria": {
                                   "in": {
                                       "SeriesDescription": 
                                       "anat_T1w_acq-MPRAGE_run"
                                   }
                                   }
                               })
# save and print the config again
writeconfig(config)
!cat config.json

In [ ]:
# take 2 - do we recognise the t1?
!dcm2bids -d bids_test5-20170120 -p 01 -c config.json -o output

In [ ]:
# now the output dir looks more interesting
!tree -I tmp_dcm2bids output

In [ ]:
# some of the required BIDS project root files have been initialised, e.g.
!cat output/dataset_description.json

# Scaling up to a complete dcm2bids conversion

In [ ]:
# house keeping
del config["descriptions"][1:]
# we need a task for valid BIDS - "rest" is handy because it means we are exempt from
# needing the events.tsv file (phantoms are at rest by definition, right?)
config["descriptions"].append({"dataType": "func", 
                               "suffix": "bold",
                               "criteria": {
                                   "in": {
                                       "PulseSequenceDetails": "bold"
                                   }
                                   },
                               "customHeader": {"TaskName": "rest"}
                               })
# unfortunately no real way to distinguish the fieldmap magnitude images without
# matching on the echo times
echo1 = 0.00492
echo2 = 0.00738
config["descriptions"].append({"dataType": "fmap", 
                               "suffix": "magnitude1",
                               "criteria": {
                                   "in": {
                                       "PulseSequenceDetails": "field_mapping"
                                   },
                                   "equal": {
                                       "ImageType":
                                       ["ORIGINAL", "PRIMARY", "M", "ND", "NORM"],
                                       "EchoTime": echo1
                                   }
                                   }
                               })
config["descriptions"].append({"dataType": "fmap", 
                               "suffix": "magnitude2",
                               "criteria": {
                                   "in": {
                                       "PulseSequenceDetails": "field_mapping"
                                   },
                                   "equal": {
                                       "ImageType":
                                       ["ORIGINAL", "PRIMARY", "M", "ND", "NORM"],
                                       "EchoNumber": 2,
                                       "EchoTime": echo2
                                   }
                                   }
                               })
# need to intervene here to set EchoTime1 and EchoTime2 since it's a phase difference
# image (see https://github.com/rordenlab/dcm2niix/issues/139)
config["descriptions"].append({"dataType": "fmap", 
                               "suffix": "phasediff",
                               "criteria": {
                                   "in": {
                                       "PulseSequenceDetails": "field_mapping"
                                   },
                                   "equal": {
                                       "ImageType": ["ORIGINAL", "PRIMARY", "P", "ND"]
                                   }
                                   },
                               "customHeader": {
                                   "EchoTime1": echo1,
                                   "EchoTime2": echo2
                               }
                               })
writeconfig(config, filename="config_complete.json")

In [ ]:
# here's one I made earlier...
!cat config_complete.json

In [ ]:
# take 3 - do we now recognise all the series we want to convert?
shutil.rmtree("output", ignore_errors=True)
!dcm2bids -d bids_test5-20170120 -p 01 -c config_complete.json -o output

In [ ]:
# the output directory looks fairly complete
!tree -I tmp_dcm2bids output

# Checking conversions with bids-validator
Also available as a handy web app

In [ ]:
# NB you would still need to add some more info manually in e.g. study_description.json
!bids-validator output

# Resources

* [The BIDS starter kit](https://github.com/bids-standard/bids-starter-kit) - wiki with links to various resources
* [The official BIDS website](http://bids.neuroimaging.io/) - the official BIDS specification is surprisingly readable
* [Michael Notter's Nipype tutorial](https://github.com/miykael/nipype_tutorial) - great interactive tutorials on working with BIDS data in Python
* [BIDS-Validator web app](https://bids-standard.github.io/bids-validator/) - convenient way to check BIDS conversions
* [dcm2bids (jooh fork)](https://github.com/jooh/Dcm2Bids) - run the conversion code presented here